In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ydata_profiling import ProfileReport

import spacy
from textblob import TextBlob


df = pd.read_csv("Emotion_final.csv")
pd.set_option('display.max_columns', None)  

c:\Users\impej\miniconda3\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\impej\AppData\Roaming\Python\Python310\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


## création de la BDD fake avec elastique search 

In [1]:
# import requests
# import json
# from faker import Faker

# url = 'http://localhost:9200/notes/_bulk'
# headers = {'Content-Type': 'application/x-ndjson'}

# faker = Faker()

# data = ''
# for _ in range(100):
#     patient_lastname = faker.last_name()
#     patient_firstname = faker.first_name()
#     text = faker.text()
#     date = faker.date()
#     patient_left = faker.boolean()
#     emotion = faker.word()
#     confidence = faker.pyfloat(left_digits=1, right_digits=2, positive=True)

#     index = {"index": {}}
#     doc = {
#         "patient_lastname": patient_lastname,
#         "patient_firstname": patient_firstname,
#         "text": text,
#         "date": date,
#         "patient_left": patient_left,
#         "emotion": emotion,
#         "confidence": confidence
#     }

#     data += json.dumps(index) + '\n'
#     data += json.dumps(doc) + '\n'

# response = requests.post(url, headers=headers, data=data)
# print(response.text)


{"took":755,"errors":false,"items":[{"index":{"_index":"notes","_type":"_doc","_id":"h5IEIIkBdEg6yxurU0ah","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":0,"_primary_term":1,"status":201}},{"index":{"_index":"notes","_type":"_doc","_id":"iJIEIIkBdEg6yxurU0ah","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1,"_primary_term":1,"status":201}},{"index":{"_index":"notes","_type":"_doc","_id":"iZIEIIkBdEg6yxurU0ah","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":2,"_primary_term":1,"status":201}},{"index":{"_index":"notes","_type":"_doc","_id":"ipIEIIkBdEg6yxurU0ah","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":3,"_primary_term":1,"status":201}},{"index":{"_index":"notes","_type":"_doc","_id":"i5IEIIkBdEg6yxurU0ah","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":4,"_primary_term":1

In [2]:
import requests
import json

url = 'http://localhost:9200/notes/_search'
headers = {'Content-Type': 'application/json'}

query = {
    "query": {
        "match_all": {}
    }
}

response = requests.get(url, headers=headers, data=json.dumps(query))
print(response.text)


{"took":4,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":100,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"notes","_type":"_doc","_id":"h5IEIIkBdEg6yxurU0ah","_score":1.0,"_source":{"patient_lastname": "Chambers", "patient_firstname": "Adrian", "text": "Source decide game player dog owner often. Tree increase particularly arrive friend stay.\nSituation Congress Democrat stand. Cold he Mr.", "date": "2008-09-07", "patient_left": true, "emotion": "break", "confidence": 5.53}},{"_index":"notes","_type":"_doc","_id":"iJIEIIkBdEg6yxurU0ah","_score":1.0,"_source":{"patient_lastname": "Davis", "patient_firstname": "Gary", "text": "Democrat factor number card people value daughter. Win product both reveal leg he fall. Kid agree through newspaper boy senior theory light.", "date": "2007-01-24", "patient_left": true, "emotion": "add", "confidence": 0.19}},{"_index":"notes","_type":"_doc","_id":"iZIEIIkBdEg6yxurU0ah","_score":1.0,"_sou

In [4]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
df = pd.read_csv("Emotion_final.csv")

nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
    filtered_tokens = [token for token in tokens if not (nlp.vocab[token].is_punct or nlp.vocab[token].is_stop or len(token) < 3 or token.isnumeric() or token.isspace() or token.lower() in {'feel', 'feeling','feelings', 'like','im','really', 'today','didnt','go', 'know', 'get', 'want', 'would', 'time', 'little', 'ive', 'still', 'even', 'one',  'people', 'think', 'bit', 'things', 'much', 'dont', 'make', 'day', 'something', 'back', 'going', 'way', 'could'})]    
    return ' '.join(filtered_tokens)

corpus = df['Text'].tolist()
corpus = [preprocess(text) for text in corpus]

# TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['000' '12' '1997' ... 'zum' 'zumba' 'zzap']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


y = df['Emotion'].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.86


In [13]:
from elasticsearch import Elasticsearch
from faker import Faker

faker = Faker()

# Prédictions de probabilité pour chaque texte
proba = clf.predict_proba(X)

# Récupération des étiquettes de classe
class_labels = le.inverse_transform(clf.classes_)

# Remplissage des colonnes "emotion" et "confidence"
df['emotion'] = [class_labels[np.argmax(p)] for p in proba]
df['confidence'] = [np.max(p) for p in proba]

# Connexion à Elasticsearch
es = Elasticsearch(hosts=['http://localhost:9200'])

# Indexation des données dans Elasticsearch
for _, row in df.iterrows():
    patient_lastname = faker.last_name()
    patient_firstname = faker.first_name()
    date = faker.date()
    patient_left = faker.boolean()
    
    doc = {
        'patient_lastname': patient_lastname,
        'patient_firstname': patient_firstname,
        'text': row['Text'],
        'date': date,
        'patient_left': patient_left,
        'emotion': row['emotion'],
        'confidence': row['confidence']
    }
    es.index(index='notes', body=doc)


C:\Users\impej\AppData\Local\Temp\ipykernel_9752\4233235036.py:35: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='notes', body=doc)
C:\Users\impej\AppData\Local\Temp\ipykernel_9752\4233235036.py:35: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='notes', body=doc)


In [11]:
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch(hosts=['http://localhost:9200'])

# Requête de recherche pour récupérer tous les documents
query = {
    "query": {
        "match": {
            "emotion": "happy"
        }
    }
}

# Exécution de la requête de recherche
res = es.search(index='notes', body=query)

# Récupération des résultats
hits = res['hits']['hits']

# Affichage des résultats
for hit in hits:
    print(hit['_source'])


{'text': 'i feel much more energetic generally im sleeping better and so is my wife', 'emotion': 'happy', 'confidence': 1.0}
{'text': 'i can however tell you that it will hurt you will be humiliated and you will feel wonderful afterwards', 'emotion': 'happy', 'confidence': 0.71}
{'text': 'ive gone through stages of nervousness and sheer terror but now i am feeling relaxed and excited', 'emotion': 'happy', 'confidence': 0.9}
{'text': 'i feel joyful and not feeble', 'emotion': 'happy', 'confidence': 0.99}
{'text': 'i feel like garbage i cant think about being thankful right now it hurts too badly', 'emotion': 'happy', 'confidence': 0.91}
{'text': 'i can genuinely say from the bottom of my heart that i feel absolutely thankful', 'emotion': 'happy', 'confidence': 0.93}
{'text': 'i see how strong and bright you are and as you meet your milestones weeks early i feel assured that my gut was always right', 'emotion': 'happy', 'confidence': 0.95}
{'text': 'i feel rewarded and useful and valuabl

C:\Users\impej\AppData\Local\Temp\ipykernel_9752\1124562407.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='notes', body=query)
C:\Users\impej\AppData\Local\Temp\ipykernel_9752\1124562407.py:16: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index='notes', body=query)


# effacement

In [12]:
# import requests

# index = 'notes'  # Remplacez par le nom de votre index

# # URL de l'API de suppression des documents
# url = f'http://localhost:9200/{index}/_delete_by_query'

# # Corps de la requête de suppression
# query = {
#     "query": {
#         "match_all": {}
#     }
# }

# # Envoyer la requête de suppression
# response = requests.post(url, json=query)

# # Vérifier la réponse
# if response.status_code == 200:
#     print("Les documents ont été supprimés avec succès.")
# else:
#     print("Une erreur s'est produite lors de la suppression des documents.")
#     print(response.text)

Les documents ont été supprimés avec succès.
